In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [2]:
st_day = datetime.strptime('2024-04-17', "%Y-%m-%d").date()
end_day = datetime.strptime("2024-05-02", "%Y-%m-%d").date()
print(st_day)
print(end_day)

2024-04-17
2024-05-02


In [3]:
def vietnamese_numerical_to_numeric(vietnamese_str):
    numerical_dict = {
        'tỷ': 10**9,
        'triệu': 10**6,
        'nghìn': 10**3,
        'đồng': 1,
        'đ': 1
    }

    value = 0
    current_value = 0
    for word in vietnamese_str.split():
        if word in numerical_dict:
            current_value *= numerical_dict[word]
            value += current_value
            current_value = 0
        else:
            current_value += float(word.replace('tỷ', '').replace('triệu', '').replace('nghìn', ''))

    return value


In [4]:
import re
df = pd.DataFrame()
while st_day <= end_day:
    data_per_day = pd.read_csv(f"../data1/house_today({st_day}).csv")
    data_per_day = data_per_day[['price','link']]
    data_per_day['link'] = data_per_day['link'].apply(lambda x: re.search(r"id([0-9]*)",x).group(1))
    data_per_day['price'] = data_per_day['price'].apply(lambda x: vietnamese_numerical_to_numeric(x)/1000000000)

    df = pd.concat([df,data_per_day])
    st_day += timedelta(days=1)

In [5]:
df.drop_duplicates(subset=['link'],inplace=True)
df.rename(columns={'price':'price(billionVND)', 'link':'id'},inplace=True)
# Replace empty strings ('') with NaN
df['id'] = pd.to_numeric(df['id'], errors='coerce')

# Drop rows with NaN values in the 'id' column
df.dropna(subset=['id'], inplace=True)

# Convert the 'id' column to integer type
df['id'] = df['id'].astype(int)

In [7]:
df.shape

(18213, 2)

In [6]:
df.to_csv("prices.csv",index=False)